# STEP 0: Prepare
---
The following is the agenda for this stage of brainmaking:
1. Import all necessary libraries
2. Obtain raw data and ferment it into a processable dataset

In [1]:
#LIBRARY IMPORT
from CyAdmin import *
print('Necessary libraries imported')

Using TensorFlow backend.


CyHome's Admin (v2) Package successfully imported
Necessary libraries imported


### Dataset description

A total of `150+` different strings used to prepare a solid dataset, with the followind trigger categories:
- **#LIGHTS_ON**
: Switch on the bulb in the room
- **#LIGHTS_OFF**
: Switch off the bulb in the room
- **#FAN_ON**
: Switch on the ceiling-fan
- **#FAN_OFF**
: Switch off the ceiling-fan
- **#DOOR_OPEN**
: Open and unlock the main door of the house
- **#DOOR_CLOSE**
: Close and lock the main door of the house
- **#CURTAINS_OPEN**
: Roll up and open the window curtains
- **#CURTAINS_CLOSE**
: Close and the window curtains

In [3]:
#VIEW RAW DATASET
raw=pd.read_csv('genDS.csv')
raw.tail(10)

,String,Trigger
234,Pull in the curtain screen,#CURTAINS_CLOSE
235,Roll down the curtains,#CURTAINS_CLOSE
236,I need only inside light,#CURTAINS_CLOSE
237,Seal the window,#CURTAINS_CLOSE
238,Fluff the curtains,#CURTAINS_CLOSE
239,Wind down the curtains,#CURTAINS_CLOSE
240,Bring in the blinds,#CURTAINS_CLOSE
241,Restrict outside view,#CURTAINS_CLOSE
242,Block outside view,#CURTAINS_CLOSE
243,Drape the windows,#CURTAINS_CLOSE


In [4]:
#ENUMERATE TRIGGER STRINGS TO NUMBERS
dicto={}
count=0
y_np=[]
for i in raw.Trigger:
    if i in dicto.keys():
        y_np.append(dicto[i])
    else:
        dicto[i]=count
        count+=1
        y_np.append(dicto[i])
revdicto={}
for i in dicto.keys():
    revdicto[dicto[i]]=i;
#print('Trigger enumeration results: ',y_np)        

### Preprocess strings
The following are the steps to convert a given string into a representative text vector

| Step number | Description | example input | example output |
| ----------- | ----------- | ----------- | ----------- |
| 01 | Decapitalise string | **On** the fans | **on** the fans |
| 02 | Remove stop words | on **the** fans | on fans |
| 03 | Stem and lemmatise | on **fans** | on **fan** |
| 04 | TF-IDF vectorise | **on fan** | an **array** with **0** and other **numbers** |

In [17]:
def aggressive_fermenterv2(s):
    
        
    s=s.lower()
    stopwords=[" the"," is"," and"," get"," to"," please"]
    for stopper in stopwords:
        s=s.replace(stopper,'')
    replace_rules=[["ness",""],["you"," "],["i am"," "],["es","e"],["sses","ss"],["ies","y"],["s",""],[","," "]]

    for rule in range(len(replace_rules)):
        regstring=replace_rules[rule][0]+r"\s+"
        endreg=replace_rules[rule][0]+r"$"
        s=re.sub(regstring,replace_rules[rule][1]+" ",s)
        s=re.sub(endreg,replace_rules[rule][1]+" ",s)
        
    
    s=re.sub(r"\s\s"," ",s)
    s=re.sub(r"^\s","",s)
    s=re.sub(r"\s$","",s)
    return s

GENTEXT=[]
pristine=[]
for sentence in raw.String:
    GENTEXT.append(aggressive_fermenterv2(sentence))
    
tfidf=TfidfVectorizer(min_df=1,norm=None,smooth_idf=False, ngram_range=(1, 2))
features=tfidf.fit_transform(GENTEXT)
pristine=pd.DataFrame(
features.todense(),
columns=tfidf.get_feature_names())
x_mat=pristine.to_numpy()
for i in range(len(pristine)):
    for j in range(len(pristine.iloc[i])):
        if(pristine.iloc[i,j]!=0):
            pristine.iloc[i,j]=1
pristine

,activate,activate fan,air,air in,air out,all,all door,all entrie,answer,answer door,...,will latch,will safeguard,will seal,wind,wind down,wind up,window,would,would slam,zero
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
240,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
tfidf.vocabulary_

{'turn': 402,
 'on': 254,
 'light': 209,
 'turn on': 407,
 'on light': 257,
 'switch': 390,
 'switch on': 393,
 'bring': 36,
 'bring light': 40,
 'light on': 213,
 'up': 422,
 'room': 314,
 'light up': 217,
 'up room': 429,
 'wake': 432,
 'wake up': 433,
 'up light': 428,
 'get': 140,
 'running': 316,
 'get light': 144,
 'light running': 215,
 'turn light': 405,
 'switch light': 391,
 'bulb': 43,
 'on bulb': 255,
 'up bulb': 423,
 'turn bulb': 403,
 'bulb on': 45,
 'in': 169,
 'bring in': 39,
 'in light': 172,
 'bring bulb': 37,
 'on up': 258,
 'light in': 211,
 'brighten': 32,
 'brighten up': 35,
 'put': 299,
 'put on': 302,
 'make': 227,
 'it': 178,
 'bright': 30,
 'make it': 228,
 'it bright': 179,
 'full': 139,
 'brighten light': 33,
 'light full': 210,
 'want': 434,
 'want wake': 441,
 'need': 238,
 'need wake': 242,
 'awake': 17,
 'enough': 115,
 'sleep': 344,
 'forday': 133,
 'enough sleep': 116,
 'sleep forday': 345,
 'time': 400,
 'it time': 182,
 'time up': 401,
 'put up': 30

In [20]:
x_np=pristine.to_numpy()
y_np=y_np
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver = 'lbfgs')
model.fit(x_np,y_np)
model.score(x_np,y_np)

0.9959016393442623

In [33]:
model.coef_

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [3., 3., 9., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# SAVE and LOAD

In [26]:
f = open(r"CyHome-review\essentials\deathblowv2.txt", "w")
weightsave=model.coef_
for i in range(len(weightsave)):
    for j in range(len(weightsave[i])):
        if(weightsave[i][j]>0):
            weightsave[i][j]=int(weightsave[i][j]*10)
        else:
            weightsave[i][j]=0
            
for i in range(len(weightsave)):
    for j in range(0,len(weightsave[i])):
        f.write(str(weightsave[i][j]))
        f.write(',')
    f.write('\n')
f.close()

In [35]:
f=open(r"CyHome-review/essentials/dicto.txt","w")
for i in (tfidf.vocabulary_.keys()):
    f.write(i)
    f.write(':')
    f.write(str(tfidf.vocabulary_[i]))
    f.write(':\n')
f.close()

In [37]:
preprocessing_dict={}
if(preprocessing_dict=={}):
        f=open(r"CyHome-review/essentials/dicto.txt","r")
        for line in f:
            vals=line.split(':')
            preprocessing_dict[vals[0]]=int(vals[1])
        f.close()
preprocessing_dict

{'turn': 402,
 'on': 254,
 'light': 209,
 'turn on': 407,
 'on light': 257,
 'switch': 390,
 'switch on': 393,
 'bring': 36,
 'bring light': 40,
 'light on': 213,
 'up': 422,
 'room': 314,
 'light up': 217,
 'up room': 429,
 'wake': 432,
 'wake up': 433,
 'up light': 428,
 'get': 140,
 'running': 316,
 'get light': 144,
 'light running': 215,
 'turn light': 405,
 'switch light': 391,
 'bulb': 43,
 'on bulb': 255,
 'up bulb': 423,
 'turn bulb': 403,
 'bulb on': 45,
 'in': 169,
 'bring in': 39,
 'in light': 172,
 'bring bulb': 37,
 'on up': 258,
 'light in': 211,
 'brighten': 32,
 'brighten up': 35,
 'put': 299,
 'put on': 302,
 'make': 227,
 'it': 178,
 'bright': 30,
 'make it': 228,
 'it bright': 179,
 'full': 139,
 'brighten light': 33,
 'light full': 210,
 'want': 434,
 'want wake': 441,
 'need': 238,
 'need wake': 242,
 'awake': 17,
 'enough': 115,
 'sleep': 344,
 'forday': 133,
 'enough sleep': 116,
 'sleep forday': 345,
 'time': 400,
 'it time': 182,
 'time up': 401,
 'put up': 30

In [39]:
f = open(r"CyHome-review/essentials/revdicto.txt", "w")
for i in revdicto.keys():
    f.write(str(i))
    f.write(':')
    f.write(revdicto[i])
    
    f.write(':\n')
    
f.close()

In [42]:
revdicto={}
f=open(r"CyHome-review/essentials/revdicto.txt","r")
for line in f:
    vals=line.split(':')
    revdicto[int(vals[0])]=vals[1]

# DL part

In [24]:
# CHEAT CODE: DELETE CELL THIS WHILE PRESENTING

#x_np=pristine.to_numpy()

brain=keras.models.Sequential()
brain.add(layers.InputLayer(input_shape=(len(x_np[0]),)))
brain.add(layers.Dense(8))

brain.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])


brain.set_weights([model.coef_.transpose(),[0, 0,0, 0,0, 0,0, 0]])
print('accuracy: ',model.score(x_np,y_np))

brain.summary()
#brain.save('taskv1.h5')

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [32]:
y_np2=np.eye(np.max(y_np) + 1)[y_np]

In [34]:
history=brain.fit(x_np2,y_np2,epochs=20,batch_size=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
237/237 [==============================] - 5s 20ms/step - loss: 11.4638 - acc: 0.0802
Epoch 2/20
237/237 [==============================] - 2s 10ms/step - loss: 12.9961 - acc: 0.0844
Epoch 3/20
237/237 [==============================] - 2s 10ms/step - loss: 13.2617 - acc: 0.1350
Epoch 4/20
237/237 [==============================] - 3s 11ms/step - loss: 13.2617 - acc: 0.1350
Epoch 5/20
237/237 [==============================] - 3s 11ms/step - loss: 13.2617 - acc: 0.1350
Epoch 6/20
237/237 [==============================] - 3s 11ms/step - loss: 13.2617 - acc: 0.1350
Epoch 7/20
237/237 [==============================] - 3s 11ms/step - loss: 13.2617 - acc: 0.1350
Epoch 8/20
237/237 [==============================] - 2s 10ms/step - loss: 13.2617 - acc: 0.1350
Epoch 9/20
237/237 [==============================] - 2s 10ms/step - loss: 13.2617 - acc: 0.1350
Epoch 10/20
237/237 [==============================] - 2s 10ms/step - loss: 13.

In [29]:
model.predict([pristine.iloc[100]])

array([3])

In [87]:
brain.get_weights()[0].transpose()

array([[ 0.16919445, -0.09522658,  0.26442102, ...,  0.10069346,
         0.10069346, -0.1194134 ],
       [-0.10288014, -0.04811342, -0.05476672, ...,  0.06708349,
         0.06708349,  0.1877499 ],
       [ 0.08987086,  0.11744276, -0.02757191, ..., -0.03222255,
        -0.03222255, -0.01284183],
       ...,
       [-0.026013  , -0.0195347 , -0.00647831, ..., -0.01732804,
        -0.01732804, -0.0060078 ],
       [-0.16751835, -0.05375179, -0.11376657, ..., -0.03672469,
        -0.03672469, -0.01997198],
       [-0.05759919, -0.01940134, -0.03819785, ..., -0.02489816,
        -0.02489816, -0.0124272 ]], dtype=float32)

In [95]:
weightsave=brain.get_weights()[0]
for i in range(len(brain.get_weights()[0])):
    for j in range(len(brain.get_weights()[0][i])):
        if(weightsave[i][j]>0):
            weightsave[i][j]=int(weightsave[i][j]*10)
        else:
            weightsave[i][j]=0

In [16]:
brain.set_weights([weightsave,brain.get_weights()[1]])

In [45]:
len(model.coef_)

8

In [46]:
minn=abs(model.coef_[0][0])
maxx=minn
for i in range(len(model.coef_)):
    minn=min(minn,min(model.coef_[i]))
    maxx=max(maxx,max(model.coef_[i]))
print(minn,maxx)

0.0 21.0


In [48]:
model.predict(np.array([x_np[0]]))

array([0])

In [18]:
maxi=max(model.predict(np.array([x_np[0]]))[0])
mini=maxi
for i in range(len(x_np)):
    maxi=max(maxi,max(brain.predict(np.array([x_np[i]]))[0]))
    mini=min(mini,max(brain.predict(np.array([x_np[i]]))[0]))
maxi,mini

(1325.0, 537.0)

In [102]:
brain=models.load_model('deathblow.h5')

In [34]:
tfidf.vocabulary_

{'turn': 402,
 'on': 254,
 'light': 209,
 'turn on': 407,
 'on light': 257,
 'switch': 390,
 'switch on': 393,
 'bring': 36,
 'bring light': 40,
 'light on': 213,
 'up': 422,
 'room': 314,
 'light up': 217,
 'up room': 429,
 'wake': 432,
 'wake up': 433,
 'up light': 428,
 'get': 140,
 'running': 316,
 'get light': 144,
 'light running': 215,
 'turn light': 405,
 'switch light': 391,
 'bulb': 43,
 'on bulb': 255,
 'up bulb': 423,
 'turn bulb': 403,
 'bulb on': 45,
 'in': 169,
 'bring in': 39,
 'in light': 172,
 'bring bulb': 37,
 'on up': 258,
 'light in': 211,
 'brighten': 32,
 'brighten up': 35,
 'put': 299,
 'put on': 302,
 'make': 227,
 'it': 178,
 'bright': 30,
 'make it': 228,
 'it bright': 179,
 'full': 139,
 'brighten light': 33,
 'light full': 210,
 'want': 434,
 'want wake': 441,
 'need': 238,
 'need wake': 242,
 'awake': 17,
 'enough': 115,
 'sleep': 344,
 'forday': 133,
 'enough sleep': 116,
 'sleep forday': 345,
 'time': 400,
 'it time': 182,
 'time up': 401,
 'put up': 30

In [38]:
revdicto

{0: '#LIGHTS_ON',
 1: '#LIGHTS_OFF',
 2: '#FAN_ON',
 3: '#FAN_OFF',
 4: '#DOOR_OPEN',
 5: '#DOOR_CLOSE',
 6: '#CURTAINS_OPEN',
 7: '#CURTAINS_CLOSE'}

In [43]:
revdicto

{0: '#LIGHTS_ON',
 1: '#LIGHTS_OFF',
 2: '#FAN_ON',
 3: '#FAN_OFF',
 4: '#DOOR_OPEN',
 5: '#DOOR_CLOSE',
 6: '#CURTAINS_OPEN',
 7: '#CURTAINS_CLOSE'}